# <center>AIM 5001 Module 11 Assignment (100 Points)

 # <center> Part 1: Tidying and Reshaping Data

In [1]:
import pandas as pd
import numpy as np

# Import the data

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/shenghanyueh/AIM-5001/main/M11/M11_Data.csv')

In [3]:
df

,Month,Category,Caltex,Gulf,Mobil
0,Open,Engine Oil,140 : 000,199 : 000,141 : 000
1,NaN,GearBox Oil,198 : 000,132 : 000,121 : 000
2,Jan,Engine Oil,170 : 103,194 : 132,109 : 127
3,NaN,GearBox Oil,132 : 106,125 : 105,191 : 100
4,Feb,Engine Oil,112 : 133,138 : 113,171 : 101
5,NaN,GearBox Oil,193 : 148,199 : 119,134 : 127
6,Mar,Engine Oil,184 : 100,141 : 141,114 : 108
7,NaN,GearBox Oil,138 : 121,172 : 133,193 : 115
8,Apr,Engine Oil,149 : 150,117 : 118,117 : 118
9,NaN,GearBox Oil,185 : 125,191 : 133,119 : 121


# 1.1 Transfer into a long tidy dataframe

__Fill the nan in month with the previous value.__

In [4]:
df['Month']=df['Month'].fillna(method='ffill')

__Reshape the table and separate the string into purchasing and consuming.__

In [5]:
ids = ["Month","Category"]
values = df.columns[2:]
df = pd.melt(frame=df, id_vars=ids, value_vars=values, var_name='Company', value_name='values')

In [6]:
df[['purchase', 'consume']] = df['values'].str.split(':', 0, expand=True)
df=df.drop(columns='values')

C:\Users\User\AppData\Local\Temp\ipykernel_6528\1709041608.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['purchase', 'consume']] = df['values'].str.split(':', 0, expand=True)


In [7]:
df

,Month,Category,Company,purchase,consume
0,Open,Engine Oil,Caltex,140,000
1,Open,GearBox Oil,Caltex,198,000
2,Jan,Engine Oil,Caltex,170,103
3,Jan,GearBox Oil,Caltex,132,106
4,Feb,Engine Oil,Caltex,112,133
5,Feb,GearBox Oil,Caltex,193,148
6,Mar,Engine Oil,Caltex,184,100
7,Mar,GearBox Oil,Caltex,138,121
8,Apr,Engine Oil,Caltex,149,150
9,Apr,GearBox Oil,Caltex,185,125


# 1.2.1 What was the amount of oil remaining for each type/brand at the end of the chronological period

__The rest of the oil (net) can be achieved by subtracting the consumption from purchasing amount.__

In [8]:
df['purchase']=df['purchase'].astype('int') # transform the string into numbers
df['consume']=df['consume'].astype('int') # transform the string into numbers
df.groupby(['Category','Company'])['purchase'].sum()-df.groupby(['Category','Company'])['consume'].sum()

Category     Company
Engine Oil   Caltex     330
             Gulf       302
             Mobil      345
GearBox Oil  Caltex     375
             Gulf       419
             Mobil      299
dtype: int32

# 1.2.2 What was the most consumed brand of oil across the two separate categories/types of oil?

In [9]:
df.groupby(['Category','Company'])['consume'].sum()

Category     Company
Engine Oil   Caltex     754
             Gulf       761
             Mobil      676
GearBox Oil  Caltex     746
             Gulf       733
             Mobil      721
Name: consume, dtype: int32

__For engine oil, Gulf consumes the most; for gearbox oil, Caltex consume the most.__

In [10]:
df.groupby(['Company'])['consume'].sum()

Company
Caltex    1500
Gulf      1494
Mobil     1397
Name: consume, dtype: int32

__In total, Caltex consumes the most.__

# 1.3 transform “long” data to “wide” format

__To know the trends of purchasing and consumption during the time for each company and type, I transfer to a 'wide' table that is different from the original one. The month would be the column. The amount of purchasing and consuming oil is separated into two sides.__ 

In [15]:
wide_data=df.pivot_table(values=['purchase','consume'],index=['Company','Category'],columns=['Month'],sort=False)

In [16]:
wide_data

purchase                               consume            \
Month                   Open  Jan  Feb  Mar  Apr  May  Jun    Open  Jan  Feb   
Company Category                                                               
Caltex  Engine Oil       140  170  112  184  149  170  159       0  103  133   
        GearBox Oil      198  132  193  138  185  168  107       0  106  148   
Gulf    Engine Oil       199  194  138  141  117  104  170       0  132  113   
        GearBox Oil      132  125  199  172  191  138  195       0  105  119   
Mobil   Engine Oil       141  109  171  114  117  200  169       0  127  101   
        GearBox Oil      121  191  134  193  119  121  141       0  100  127   

                                         
Month                Mar  Apr  May  Jun  
Company Category                         
Caltex  Engine Oil   100  150  139  129  
        GearBox Oil  121  125  117  129  
Gulf    Engine Oil   141  118  119  138  
        GearBox Oil  133  133  102  141  
Mobil   Engine Oil   108  118  117  105  
        GearBox Oil  115  121  146  112

__We know the trends of two types of oil management in each company through the above table.__

# <center>Part 2: Using Your GroupBy and Data Aggregation Skills

# Import the data

In [17]:
df=pd.read_csv('https://raw.githubusercontent.com/shenghanyueh/AIM-5001/main/M11/bridges.data.version1.csv',header=None)

In [18]:
df=df.loc[:,0].str.split(',', 0, expand=True)

C:\Users\User\AppData\Local\Temp\ipykernel_6528\489360034.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df=df.loc[:,0].str.split(',', 0, expand=True)


In [19]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,E1,M,3,1818,HIGHWAY,?,2,N,THROUGH,WOOD,SHORT,S,WOOD
1,E2,A,25,1819,HIGHWAY,1037,2,N,THROUGH,WOOD,SHORT,S,WOOD
2,E3,A,39,1829,AQUEDUCT,?,1,N,THROUGH,WOOD,?,S,WOOD
3,E5,A,29,1837,HIGHWAY,1000,2,N,THROUGH,WOOD,SHORT,S,WOOD
4,E6,M,23,1838,HIGHWAY,?,2,N,THROUGH,WOOD,?,S,WOOD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,E84,A,24,1969,HIGHWAY,870,6,G,THROUGH,STEEL,MEDIUM,F,ARCH
104,E91,O,44,1975,HIGHWAY,3756,6,G,THROUGH,STEEL,LONG,F,ARCH
105,E90,M,7,1978,HIGHWAY,950,6,G,THROUGH,STEEL,LONG,F,ARCH
106,E100,O,43,1982,HIGHWAY,?,?,G,?,?,?,F,?


In [20]:
colname=['IDENTIF',
         'RIVER',
         'LOCATION',
         'ERECTED',
         'PURPOSE',
         'LENGTH',
         'LANES',
         'CLEAR',
         'T_OR_D',
         'MATERIAL',
         'SPAN',
         'REL',
         'TYPE']

In [21]:
df.columns=colname

In [22]:
df

,IDENTIF,RIVER,LOCATION,ERECTED,PURPOSE,LENGTH,LANES,CLEAR,T_OR_D,MATERIAL,SPAN,REL,TYPE
0,E1,M,3,1818,HIGHWAY,?,2,N,THROUGH,WOOD,SHORT,S,WOOD
1,E2,A,25,1819,HIGHWAY,1037,2,N,THROUGH,WOOD,SHORT,S,WOOD
2,E3,A,39,1829,AQUEDUCT,?,1,N,THROUGH,WOOD,?,S,WOOD
3,E5,A,29,1837,HIGHWAY,1000,2,N,THROUGH,WOOD,SHORT,S,WOOD
4,E6,M,23,1838,HIGHWAY,?,2,N,THROUGH,WOOD,?,S,WOOD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,E84,A,24,1969,HIGHWAY,870,6,G,THROUGH,STEEL,MEDIUM,F,ARCH
104,E91,O,44,1975,HIGHWAY,3756,6,G,THROUGH,STEEL,LONG,F,ARCH
105,E90,M,7,1978,HIGHWAY,950,6,G,THROUGH,STEEL,LONG,F,ARCH
106,E100,O,43,1982,HIGHWAY,?,?,G,?,?,?,F,?


In [23]:
df.describe()

,IDENTIF,RIVER,LOCATION,ERECTED,PURPOSE,LENGTH,LANES,CLEAR,T_OR_D,MATERIAL,SPAN,REL,TYPE
count,108,108,108,108,108,108,108,108,108,108,108,108,108
unique,108,4,55,70,4,65,5,3,3,4,4,4,8
top,E1,A,39,1945,HIGHWAY,?,2,G,THROUGH,STEEL,MEDIUM,F,SIMPLE-T
freq,1,49,5,6,71,27,61,80,87,79,53,58,44


# 2.1

__The groupby function can separate the data set into subgroups. Using the count function will demonstrate the number of recordings in each subgroup.__

In [24]:
res=df.groupby(['RIVER','PURPOSE','MATERIAL']).count().iloc[:,0]
res=pd.DataFrame(res.rename('How Many?'))

In [25]:
res

How Many?
RIVER PURPOSE  MATERIAL           
A     AQUEDUCT IRON              1
               WOOD              3
      HIGHWAY  ?                 1
               IRON              2
               STEEL            21
               WOOD              8
      RR       IRON              1
               STEEL             9
               WOOD              2
      WALK     STEEL             1
M     HIGHWAY  IRON              4
               STEEL            19
               WOOD              3
      RR       IRON              2
               STEEL            13
O     HIGHWAY  ?                 1
               IRON              1
               STEEL             9
      RR       STEEL             4
Y     HIGHWAY  STEEL             2
      RR       STEEL             1

# 2.2

__Transfer string '?' into nan.__

In [26]:
df['LENGTH']=df['LENGTH'].replace({'?':np.nan})

__Check the result.__

In [27]:
df['LENGTH'].isna().sum()

27

In [28]:
# transfer data type
df['LENGTH']=df['LENGTH'].astype('float')

# After separating into subgroups, use mean() to generate the average value. 
# By selecting column 'LENGTH', we can generate the following table.
res=pd.DataFrame(df.groupby(['PURPOSE','MATERIAL']).mean()['LENGTH'].rename('Average Length'))

C:\Users\User\AppData\Local\Temp\ipykernel_6528\1168177965.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res=pd.DataFrame(df.groupby(['PURPOSE','MATERIAL']).mean()['LENGTH'].rename('Average Length'))


In [29]:
res

Average Length
PURPOSE  MATERIAL                
AQUEDUCT IRON         1000.000000
         WOOD         1092.000000
HIGHWAY  ?                    NaN
         IRON         1216.666667
         STEEL        1557.804348
         WOOD         1053.375000
RR       IRON         1100.000000
         STEEL        1946.850000
         WOOD                 NaN
WALK     STEEL                NaN

# 2.3

In [30]:
df['ERECTED']=df['ERECTED'].replace({'?':np.nan})
# Transfer the data type
df['ERECTED']=df['ERECTED'].astype('int')

In [31]:
# Using cut function to separate into 4 subgroups according to numerical numbers. 
# The function agg will aggregate multiple statistics results

res=df.groupby(pd.cut(df.ERECTED, 4,precision=0))['LENGTH'].agg(['mean','count','max','min']) 
res=pd.DataFrame(res)

# rename the columns
res.columns=['Average Length','Count','Max Length','Min Length']

In [32]:
res

,Average Length,Count,Max Length,Min Length
ERECTED,,,,
"(1818.0, 1860.0]",1094.625000,8,1500.0,990.0
"(1860.0, 1902.0]",1603.347826,23,4558.0,1000.0
"(1902.0, 1944.0]",1676.181818,33,3000.0,860.0
"(1944.0, 1986.0]",1530.411765,17,3756.0,804.0
